In [2]:
import tensorflow as tf
import sonnet as snt
import numpy as np
import pandas as pd

import sys
sys.path.append('/Users/andrew/Desktop/sudoku/src/sudoku')

from board import Board
from solutions import Solutions
import utils

/Users/andrew/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
# set random seed to 0
np.random.seed(0)
tf.set_random_seed(0)

In [4]:
datapath = '/Users/andrew/Desktop/sudoku/data/'

In [5]:
solutions = Solutions(datapath + 'solutions5.txt')

In [6]:
puzzles = solutions.get_puzzles_by_hints()

In [7]:
inputs = []
outputs = []
for i in range(4, 9):
    for board in puzzles[i]:
        inputs.append(board)
        outputs.append(solutions[board])

In [21]:
def vector_encode(board_string):
    dim_x, dim_y, board = board_string.split('.')
    max_digit = int(dim_x) * int(dim_y)
    vector = np.zeros((max_digit*len(board),), dtype=np.float64)
    for i in range(len(board)):
        if board[i] != '0':
            vector[i*max_digit + int(board[i]) - 1] = 1
        else:
            vector[i*max_digit:(i+1)*max_digit] = 1/max_digit
    return vector

In [68]:
class Solver(snt.AbstractModule):
    """Docstring for Solver."""
    def __init__(self, max_digit, hidden_sizes, name="solver"):
        """Docstring explaining __init__ args, including types and defaults."""
        super(Solver, self).__init__(name=name)
        self.max_digit = max_digit
        self.hidden_sizes = hidden_sizes
        self.nonlinearity = tf.sigmoid
        
    # Following on from code snippet above..
    def _build(self, inputs):
        """Compute output Tensor from input Tensor."""
        last_layer = inputs
        for i in range(len(self.hidden_sizes)):
            last_layer = snt.Linear(output_size=self.hidden_sizes[i], name="h{0}".format(i))(last_layer)
            last_layer = self.nonlinearity(last_layer)
        penultimate_layer = last_layer
        cell_values = []
        for i in range(self.max_digit**2):
            last_layer = snt.Linear(output_size=self.max_digit, name="h_to_o{0}".format(i))(penultimate_layer)
            last_layer = tf.nn.softmax(last_layer)
            cell_values.append(last_layer)
        output = tf.convert_to_tensor(cell_values, dtype=tf.float32)
        output = tf.reshape(output, (1, self.max_digit**3))
        return output

In [70]:
solver = Solver(4, [128])
x = np.array([vector_encode(inputs[0].stringify())])
x = tf.convert_to_tensor(x, np.float32)
solver(x)
loss = dataset_valid.cost(valid_output_sequence_logits,
                                  valid_target_sequence)

<tf.Tensor 'solver_12/Reshape:0' shape=(1, 64) dtype=float32>

In [71]:
solver.train?
#.train(input_fn=lambda: my_input_fn(FILE_TRAIN, True, 500))

Object `solver.train` not found.
